In [169]:
import openpyxl   
import pandas as pd
import numpy as np
from pandas import read_excel
import matplotlib.pyplot as plt
import scipy.optimize as scp

In [285]:
raw_data=pd.read_excel("crone3.xlsx", engine = 'openpyxl') 
data = pd.DataFrame(raw_data)
data = data.drop([0,1]).reset_index(drop=True)
data

,Unnamed: 0,Sample,T [s],ch1 [V],ch2 [V],mean [V],inverted [V]
0,2,2,4.000000e-08,0.059089,-0.019831,0.056090,0.110644
1,3,3,6.000000e-08,0.058488,2.806148,0.056323,0.111090
2,4,4,8.000000e-08,0.057286,0.069410,0.056337,0.098479
3,5,5,1.000000e-07,0.055784,-0.128904,0.056361,0.102886
4,6,6,1.200000e-07,0.054882,-0.118989,0.056291,0.087573
...,...,...,...,...,...,...,...
131065,131067,131067,2.621340e-03,0.019830,-0.029747,0.021171,-0.029582
131066,131068,131068,2.621360e-03,0.019830,-0.009916,0.021174,-0.026134
131067,131069,131069,2.621380e-03,0.020631,-0.019831,0.021180,-0.030358
131068,131070,131070,2.621400e-03,0.021232,-0.009916,0.021171,-0.027620


In [286]:
time_column = data.columns[2]
ch1_column = data.columns[3]
data_column = data.columns[-1]
print(time_column,ch1_column,data_column)
#print(np.average(data[ch1_column])*10)

%matplotlib widget
plt.plot(data[time_column],data[ch1_column])
plt.plot(data[time_column],data[data_column], color='r')
plt.show()


T [s] ch1 [V] inverted [V]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [287]:
tester = []
for i in range(len(data[ch1_column])):
    tester.append(data[ch1_column].to_numpy()[i]-data[ch1_column].to_numpy()[i-1])

thershold =  (np.max(tester)-np.average(tester))/3.5
print(thershold)

%matplotlib widget
plt.plot(data[time_column], tester)
plt.axhline(thershold, color='r')
#plt.plot(data[time_column],data[data_column], color='r')

0.010758995714285716


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [83]:
def f(x,a,b):
    return a*np.exp(-x/b)
    

In [288]:
points_candidates = np.argwhere(tester>thershold)
points_candidates = np.append(points_candidates,[[(len(data[ch1_column])-1)]], axis=0)
#points_candidates.append()

i = 0
points = []
while i<(len(points_candidates)-1):
    distance = points_candidates[i+1][0]-points_candidates[i][0]
    if distance < 200:
        if data[ch1_column][points_candidates[i][0]]>data[ch1_column][points_candidates[i+1][0]]:
            points.append(points_candidates[i][0])
        else:
            points.append(points_candidates[i+1][0])
        i = i + 1

    else:
        points.append(points_candidates[i][0])

    i = i + 1

points.append(len(data[ch1_column])-1)
#print(points)
#print(points_candidates)


%matplotlib widget
plt.plot(data[time_column],data[data_column])


for i in range(len(points)):
    plt.axvline(x= data[time_column][points[i]], color = 'r')

"""

for i in range(len(points_candidates)):
    plt.axvline(x= data[time_column][points_candidates[i][0]], color = 'g')

"""

plt.show()




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [297]:

thaus = []

#for i in [10]:

for i in range(len(points)-1):

    indexes = np.arange(points[i],points[i+1])[:-20]
    val_between_imp = data[data_column].to_numpy()[indexes]
    peak = np.argmax(val_between_imp)

    times_fit = np.array(data[time_column ].to_numpy()[indexes][peak:] - data[time_column ].to_numpy()[indexes][peak:][0],dtype=np.float32)
    val_fit = np.array((data[data_column].to_numpy()[indexes][peak:]-np.average(data[data_column].to_numpy()[indexes][peak:][-20:])),dtype=np.float32)
    a_init = val_fit[0]
    b_init = 0.000000146
    #print(-1/(np.log(val_fit[80]/a_init)/times_fit[180]))

    params, pcov = scp.curve_fit(f, times_fit, val_fit, [a_init,b_init])
    err = np.sqrt(np.diag(pcov))

    thaus.append(params[1])
    #print(params[1])
    
    #"""
    """
    %matplotlib widget
    plt.plot(times_fit,val_fit)
    plt.plot(times_fit, f(times_fit, params[0], params[1]), color = 'r')
    """
    #"""
    
print(thaus)
print(np.average(thaus),np.std(thaus))






[1.937216015982185e-06, 2.0432586067587988e-06, 2.068432974889831e-06, 1.928714584596575e-06, 1.977639654146446e-06, 2.0028054021474862e-06, 1.9725451359596033e-06, 2.0630638383874398e-06, 1.9622704801764614e-06, 2.06824791820316e-06, 2.035057714145749e-06, 2.061988838132744e-06, 1.987886179178823e-06, 2.0032173329051787e-06, 2.0516848697393927e-06, 2.0427067937717308e-06, 1.981154311737329e-06, 1.9935364474678356e-06, 1.960198422037959e-06, 2.022688200575031e-06, 2.004731404596017e-06, 2.053741567865447e-06, 1.93550706819286e-06, 2.0744240070626113e-06, 1.8740014655299086e-06, 1.974020563806978e-06, 2.080466463201298e-06, 1.9588366815586557e-06]
2.0042872479554833e-06 5.2075352829104503e-08


nitrogen:
2.3585670231162744e-06 6.909846269625634e-08
air1:
2.324143924882319e-06 7.554601986339205e-08
air2:
2.1719835111826933e-06 2.170831117867877e-07
crone1:
1.238491436448836e-06 8.084166046204459e-08
crone2:
5.165562411321616e-07 2.637642935183529e-08
crone3:
2.0042875401838618e-06 5.207566384658046e-08
spark1:
2.2673826825520517e-07 9.350318289748961e-08
spark2:
1.455335452280397e-07 2.6910387115750422e-08


